In [ ]:
## Deep Learning Experiments – Food Spoilage Detection

# This notebook contains CNN-based experiments conducted in Google Colab
# for academic comparison with the deployed machine learning model.

In [ ]:
import os
import shutil

os.makedirs("/content/dataset", exist_ok=True)

if not os.path.exists("/content/dataset/train"):
    shutil.copytree(
        "/content/drive/MyDrive/train",
        "/content/dataset/train"
    )
    print("Dataset copied to runtime")
else:
    print("Dataset already exists")


In [ ]:
import os
os.listdir("/content/dataset/train")


['fresh', 'rotten']

In [ ]:
#STEP 1: Building the CNN Model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt

#tf.config.list_physical_devices('GPU')

In [ ]:
# STEP 2: Data Generators

img_size = (100, 100)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    "/content/dataset/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="training"
)

val_data = datagen.flow_from_directory(
    "/content/dataset/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation"
)


Found 8722 images belonging to 2 classes.
Found 2180 images belonging to 2 classes.


In [ ]:
# STEP 3: Defining CNN Architecture

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(100,100,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# STEP 4: Compiling Model

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# STEP 5: Training (10 epochs only)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 56s 183ms/step - accuracy: 0.7001 - loss: 0.5613 - val_accuracy: 0.8197 - val_loss: 0.4073
Epoch 2/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 46s 167ms/step - accuracy: 0.8781 - loss: 0.2785 - val_accuracy: 0.8734 - val_loss: 0.3118
Epoch 3/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 45s 166ms/step - accuracy: 0.9031 - loss: 0.2269 - val_accuracy: 0.8807 - val_loss: 0.2940
Epoch 4/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 46s 168ms/step - accuracy: 0.9285 - loss: 0.1800 - val_accuracy: 0.9151 - val_loss: 0.2399
Epoch 5/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 46s 168ms/step - accuracy: 0.9397 - loss: 0.1509 - val_accuracy: 0.9096 - val_loss: 0.2913
Epoch 6/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 45s 167ms/step - accuracy: 0.9471 - loss: 0.1353 - val_accuracy: 0.9083 - val_loss: 0.2987
Epoch 7/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 46s 167ms/step - accuracy: 0.9544 - loss: 0.1164 - val_accuracy: 0.8807 - val_loss: 0.3882
Epoch 8/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 46s 170ms/step - accuracy: 0.9578 - loss: 0

In [ ]:
# STEP 6: Evaluate Performance

val_loss, val_acc = model.evaluate(val_data)
print("Validation Accuracy:", val_acc)


69/69 ━━━━━━━━━━━━━━━━━━━━ 11s 152ms/step - accuracy: 0.9207 - loss: 0.3156
Validation Accuracy: 0.918807327747345


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 33856)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,166,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,558,917 (25.02 MB)

 Trainable params: 2,186,305 (8.34 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,372,612 (16.68 MB)

In [ ]:
# Step 7: Confusion Matrix & Report

val_data = val_gen.flow_from_directory(
    "/content/dataset/train",
    target_size=(100, 100),   # ✅ FIXED
    batch_size=32,
    class_mode="binary",
    shuffle=False
)


Found 10902 images belonging to 2 classes.


In [ ]:
# Step 7 continuation

val_data.reset()

y_pred_probs = model.predict(val_data)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()
y_true = val_data.classes

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))


341/341 ━━━━━━━━━━━━━━━━━━━━ 45s 132ms/step
[[4688   53]
 [ 329 5832]]
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      4741
           1       0.99      0.95      0.97      6161

    accuracy                           0.96     10902
   macro avg       0.96      0.97      0.96     10902
weighted avg       0.97      0.96      0.97     10902



In [ ]:
# STEP 8: Saving Model

from google.colab import drive
drive.mount('/content/drive')

model.save("/content/drive/MyDrive/cnn_fruit_spoilage_model.h5")

## How to load later (for app / demo)
#from google.colab import drive
#drive.mount('/content/drive')
#model.save("/content/drive/MyDrive/cnn_fruit_spoilage_model.h5")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
